In [1]:
%%capture
!pip install llama-index-llms-huggingface
!pip install llama-index
!pip install llama-index-llms-huggingface llama-index-embeddings-huggingface
!pip install llama-index-llms-openai

In [2]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [3]:
!mkdir -p "data/chess/"
!wget "http://dspace.vnbrims.org:13000/jspui/bitstream/123456789/4980/1/Chess_%20The%20Complete%20Guide%20To%20Chess%20-%20Master_%20Chess%20Tactics%2C%20Chess%20Openings%2C%20and%20Chess%20Strategies.pdf" -O 'data/chess/chess.pdf'

--2024-05-26 17:37:56--  http://dspace.vnbrims.org:13000/jspui/bitstream/123456789/4980/1/Chess_%20The%20Complete%20Guide%20To%20Chess%20-%20Master_%20Chess%20Tactics%2C%20Chess%20Openings%2C%20and%20Chess%20Strategies.pdf
Resolving dspace.vnbrims.org (dspace.vnbrims.org)... 124.108.19.235
Connecting to dspace.vnbrims.org (dspace.vnbrims.org)|124.108.19.235|:13000... connected.
HTTP request sent, awaiting response... 200 
Length: 3003524 (2.9M) [application/pdf]
Saving to: ‘data/chess/chess.pdf’

data/chess/chess.pd 100%[===================>]   2.86M  1.65MB/s    in 1.7s    

2024-05-26 17:37:59 (1.65 MB/s) - ‘data/chess/chess.pdf’ saved [3003524/3003524]



In [4]:
documents = SimpleDirectoryReader("./data/chess/").load_data()

In [16]:
from llama_index.core import PromptTemplate
# taken from https://huggingface.co/Writer/camel-5b-hf
query_wrapper_prompt = PromptTemplate(
    "Below is an instruction that describes a task. "
    "Write a response that appropriately completes the request.\n\n"
    "### Instruction:\n{query_str}\n\n### Response:"
)

In [17]:
import torch

llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.25, "do_sample": False},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="Writer/camel-5b-hf",
    model_name="Writer/camel-5b-hf",
    device_map="auto",
    tokenizer_kwargs={"max_length": 2048},
    model_kwargs={"torch_dtype": torch.float16}
)

Settings.chunk_size = 512
Settings.llm = llm

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/787 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/25.4k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

In [23]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
index = VectorStoreIndex.from_documents(documents)

In [25]:
query_engine = index.as_query_engine(streaming=True)

In [26]:
response_stream = query_engine.query("Often we disregrad chess as what type of people?")
response_stream.print_response_stream()

Token indices sequence length is longer than the specified maximum sequence length for this model (1096 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.25` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chess is played by people from various backgrounds, including humans, animals, and even machines.

In [27]:
response_stream = query_engine.query("What's the name of the Author")
response_stream.print_response_stream()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Logan Donovan

In [28]:
response_stream = query_engine.query("What is algebraic notation?")
response_stream.print_response_stream()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Algebraic notation is a method of representing chess moves using algebraic symbols and expressions, simplifying the process of learning chess moves and understanding the game's rules.

In [30]:
response_stream = query_engine.query("The knight starts by standing next to the rook on both sides and moves in an L shape fashion. This means what?")
response_stream.print_response_stream()

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.25` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The knight starts by standing next to the rook on both sides and moves in an L shape fashion. This means it takes two straight steps and one step to the right or left. Similarly, it can also move one step to the right or left and then two steps for forward or backward.

In [31]:
response_stream = query_engine.query("Great! based on the last question, if my knight is on c1, is d3 a legal move?")
response_stream.print_response_stream()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Yes, d3 is a legal move.

In [32]:
response_stream = query_engine.query("Great! based on the last question, if my knight is on c1, is d4 a legal move?")
response_stream.print_response_stream()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Yes, d4 is a legal move.

In [34]:
response_stream = query_engine.query("how d4 is a legal move for a knight that's on c1?")
response_stream.print_response_stream()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


d4 is a legal move for a knight that's on c1 because it's a valid square for the knight to move to in the opening moves of the game.

## OpenAI

In [8]:
import os
os.environ["OPENAI_API_KEY"] = "-------------------------"

In [14]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("./data/chess").load_data()
index = VectorStoreIndex.from_documents(documents)

In [24]:
query_engine = index.as_query_engine()
response = query_engine.query("but c1 and d1 squares are not diagonal for a bishop to be legal move")
print(response)

A bishop can only move diagonally and cannot move in any other direction. The squares c1 and d1 are not diagonal for a bishop to make a legal move.


## Trying prompt

In [6]:
qa_prompt_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)

In [7]:
from llama_index.core import ChatPromptTemplate

# Text QA Prompt
chat_text_qa_msgs = [
    (
        "system",
        "Always answer the question, even if the context isn't helpful.",
    ),
    ("user", qa_prompt_str),
]
text_qa_template = ChatPromptTemplate.from_messages(chat_text_qa_msgs)

In [9]:
from llama_index.llms.openai import OpenAI
documents = SimpleDirectoryReader("./data/chess/").load_data()

# Create an index using a chat model, so that we can use the chat prompts!
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

index = VectorStoreIndex.from_documents(documents)

In [14]:
print(index.as_query_engine(llm=llm).query("Who is the 2021 F1 champion?"))

I cannot provide an answer to that question based on the context information provided.


In [15]:
print(
    index.as_query_engine(
        text_qa_template=text_qa_template,
        llm=llm,
    ).query("Who is the 2021 F1 champion?")
)

The 2021 Formula 1 champion is Max Verstappen.


In [16]:
print(
    index.as_query_engine(
        text_qa_template=text_qa_template,
        llm=llm,
    ).query("what are legal moves of a knight?")
)

The legal moves of a knight in chess involve moving in an "L" shape fashion. This means the knight can either take two straight steps and then one to the right or left, or move one to the right or left and then two steps forwards or backwards. The knight can jump over pieces and capture a piece only if the victim lies on the last square of its movement.


In [17]:
print(
    index.as_query_engine(
        text_qa_template=text_qa_template,
        llm=llm,
    ).query("great! based on your last answer if a knight in on c1, is kf7 a legal move? I'm using algebraic notation")
)

No, the move Kf7 is not a legal move for a knight on c1 in algebraic notation.
